In [12]:
from analysis.tax import AutoTax

t = AutoCGT()
t.fy_view(yr_end=2019)


Capital gains for 	FY2018-2019
Total CG:		 $ 2584.02
Total CGTaxable:	 $ 2375.34
(Uses LIFO method)
        


,CapitalGains,CapitalGainsTaxable
Ticker,,
A2M,1302.000046,1262.000046
AMT,750.000216,710.000216
MSB,283.919971,253.919971
OZL,37.359916,-11.320042
QBE,59.070102,39.070102
SRS,151.664968,121.664968
